#<<----<strong>Transfer Files between different Cloud Drives using rclone

#RESUME Supported :)
#Mega, OneDrive, Google Drive, Shared Drive, etc..
#You can use others too :)</strong>

#Subscribe: <a href="https://www.youtube.com/boostupstation">BoostUpStation</a>

In [ ]:
#@title <<----<strong>Install rClone</strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
RCLONE = True #@param {type:"boolean"}
#@markdown Check AUTO_RECONNECT to prevent notebook from disconnecting!

from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True) 
  
if RCLONE==True:
  !curl https://rclone.org/install.sh | sudo bash

if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

In [ ]:
#@title <-----<strong>Upload your  Rclone.config file</strong>
def moveConfig():
  !mv rclone.conf /root/.config/rclone/rclone.conf

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
moveConfig()
print("Moved rclone.conf to /root/.config/rclone/rclone.conf")

# <center>Rclone Mount/Unmount/Copy </center>
<center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>

In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
##########################################

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
content = open("/root/.config/rclone/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
cache_path="/content/temp/rCloneTemp"
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    !rclone mount $mountNam.value: $mPoint --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir '/content/temp/rCloneTemp' --allow-other --daemon 

    if os.path.isdir(mPoint)== True:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
 
#@markdown <center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>
 
#@markdown ---
source = "" #@param {type:"string"}
destination = "" #@param {type:"string"}
mode = "copy" #@param ["copy", "check"]
dry_run = False #@param {type:"boolean"}
Extra_args = "" #@param {type:"string"}
#@markdown ---
Email_notification = False #@param {type:"boolean"}
logs = False #@param {type:"boolean"}
emailID = "" #@param {type:"string"}
password = "" #@param {type:"string"}
Receiver_ID = "" #@param {type:"string"}
#@markdown ---
import smtplib 
########################################
args = "--transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40 "
if mode == "check":
  args += "--one-way "
if dry_run == True:
    args +=" --dry-run "
if logs == True:
  args += " --log-file rClone_log.txt "
if Extra_args != "":
  args +=Extra_args
######################################
def runrClone():
  !rclone --user-agent "Mozilla" "$mode" "$source" "$destination" $args
  
def checkEmail():
  if (Email_notification == True and logs == False):
    print("You will receive only a notification after task has finished.")
    runrClone()
    #Send only notification
     
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(emailID , password)
    message = "Your rClone task has Completed!"
    s.sendmail(emailID, Receiver_ID, message)
    print("Email Alert Sent!")  
    s.quit()
  elif (Email_notification == True and logs == True):
    print("You will receive a notification with log attached after task has finished.")
    runrClone()
    #Sending email notification with logs
    
    from email.mime.multipart import MIMEMultipart 
    from email.mime.text import MIMEText 
    from email.mime.base import MIMEBase 
    from email import encoders 

    fromaddr = emailID
    toaddr = Receiver_ID
    msg = MIMEMultipart() 
    # storing the senders email address 
    msg['From'] = fromaddr 
    # storing the receivers email address 
    msg['To'] = toaddr 
    # storing the subject 
    msg['Subject'] = "Colab has Finished Running your Cell"
    # string to store the body of the mail 
    body = "Your rClone Task has Completed! Visit link to view : https://colab.research.google.com/github/SKGHD/Handy/blob/master/HandBrakeCLI_Colab.ipynb"

    msg.attach(MIMEText(body, 'plain'))
    filename = "rClone_log.txt"
    attachment = open("/content/rClone_log.txt", "rb")
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((attachment).read()) 
    # encode into base64 
    encoders.encode_base64(p) 
 
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
    msg.attach(p)  
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(fromaddr, password) 
    text = msg.as_string() 
    s.sendmail(fromaddr, toaddr, text)
    print("Email Alert Sent with log!")  
    s.quit() 
  elif (Email_notification == False and logs == False):
    print("You will not receive any notification!!!.")
    runrClone()
checkEmail()